In [1]:
from config import DATA_PATH
from preloading import load_datasets

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


# Load dataset

In [2]:
train_df, test_df = load_datasets(f"{DATA_PATH}")

load_datasets Start
loading 40000 train records
loading 10000 test records
load_datasets Start


In [68]:
train_df.head(8)

,review,sentiment,target
0,"If you enjoy films like American Pie, Road Tri...",negative,0
1,After having seen a lot of Greek movies I feel...,positive,1
2,Seeing as Keifer Sutherland plays my favorite ...,positive,1
3,My Take: A tired formula Christmas comedy. The...,negative,0
4,It's been a long time since I last saw a movie...,negative,0
5,"Over the past century, there have been many ad...",positive,1
6,Maybe television will be as brutal one day. Ma...,negative,0
7,This film was not only one of John Ford's own ...,positive,1


In [17]:
train_df.columns

Index(['review', 'sentiment'], dtype='object')

In [20]:
train_df['target'] = pd.factorize(train_df['sentiment'], sort=True)[0]

In [21]:
train_df.head()

,review,sentiment,target
0,"If you enjoy films like American Pie, Road Tri...",negative,0
1,After having seen a lot of Greek movies I feel...,positive,1
2,Seeing as Keifer Sutherland plays my favorite ...,positive,1
3,My Take: A tired formula Christmas comedy. The...,negative,0
4,It's been a long time since I last saw a movie...,negative,0


In [12]:
test_df.head()

,review,sentiment
0,A lot of the comments seem to treat this film ...,positive
1,A young man kills a young woman for no reason....,negative
2,Four macho rough'n'tumble guys and three sexy ...,positive
3,If it had not been for Christopher Guest's hil...,negative
4,After a lively if predictable opening bank-hei...,negative


In [25]:
train_df['sentiment'].value_counts()

positive    20048
negative    19952
Name: sentiment, dtype: int64

In [26]:
test_df['sentiment'].value_counts()

negative    5048
positive    4952
Name: sentiment, dtype: int64

In [4]:
import torch
import torchtext


In [5]:
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english") ## We'll use tokenizer available from PyTorch
tokenizer("Hello, How are you?")

['hello', ',', 'how', 'are', 'you', '?']

In [ ]:
    # url = {
    #     '42B': 'http://nlp.stanford.edu/data/glove.42B.300d.zip',
    #     '840B': 'http://nlp.stanford.edu/data/glove.840B.300d.zip',
    #     'twitter.27B': 'http://nlp.stanford.edu/data/glove.twitter.27B.zip',
    #     '6B': 'http://nlp.stanford.edu/data/glove.6B.zip',
    # }

In [10]:
from torchtext.vocab import GloVe
global_vectors = GloVe(name='twitter.27B', dim=100)

In [11]:
embeddings = global_vectors.get_vecs_by_tokens(tokenizer("Hello, How are you?"), lower_case_backup=True)
embeddings.shape

torch.Size([6, 100])

In [21]:
global_vectors.get_vecs_by_tokens(["am", "Enoti", "hello"], lower_case_backup=True)

tensor([[ 3.0055e-01,  8.0686e-01, -2.9994e-01, -2.8720e-01, -1.8295e-01,
          1.0378e+00,  8.7783e-02,  1.5620e-01,  1.6044e-01, -2.1472e-02,
         -3.0117e-01, -1.6142e-01, -3.9187e+00,  1.1904e-01,  9.7026e-02,
         -2.3708e-01,  5.1175e-01, -1.7012e-01,  3.1123e-01, -6.5338e-01,
          2.5613e-01,  9.6710e-02, -2.5963e-01, -2.4629e-01, -2.1014e-03,
         -1.0390e+00, -1.1399e-01, -3.3197e-01,  2.1347e-01,  3.7158e-01,
          2.1336e-01, -1.1778e-01, -9.3767e-02, -3.2307e-01,  1.8892e-01,
          1.5611e-01,  8.0243e-02,  7.2999e-02, -6.2431e-02,  4.1545e-01,
         -1.8762e+00, -7.8831e-02,  7.7317e-01, -1.2493e-01, -3.6049e-02,
          1.3602e-01,  9.1399e-02, -3.2078e-01, -4.7738e-01,  2.5548e-01,
         -2.8349e-01, -1.2392e-02,  9.2535e-02, -9.2562e-01,  2.4657e-01,
          2.9753e-03, -4.1262e-02, -1.1225e+00,  1.0138e-01,  8.8375e-02,
         -1.0234e-02, -1.3363e-01,  7.7226e-01, -4.4108e-01,  4.3879e-01,
         -2.7009e-01, -1.7261e-01,  6.

In [135]:
max_words = 25
embed_len = 100

def vectorize_batch(batch):
    X, Y = list(zip(*batch))
    X = [tokenizer(x) for x in X]
    X = [tokens+[""] * (max_words-len(tokens))  if len(tokens)<max_words else tokens[:max_words] for tokens in X]
    X_tensor = torch.zeros(len(batch), max_words, embed_len)
    for i, tokens in enumerate(X):
        X_tensor[i] = global_vectors.get_vecs_by_tokens(tokens)
    return X_tensor.reshape(len(batch), -1), torch.tensor(Y)

In [ ]:
# Pytorch Torchtext Tutorial 1: Custom Datasets and loading JSON/CSV/TSV files
# https://www.youtube.com/watch?v=KRgq4VnCr7I
# https://towardsdatascience.com/custom-datasets-in-pytorch-part-2-text-machine-translation-71c41a3e994e

# How to build custom Datasets for Text in Pytorch
# https://www.youtube.com/watch?v=9sHcLvVXsns

In [24]:
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# TODO: Change TF IDF to glove

In [136]:
class ClassificationDataset(Dataset):

    def __init__(self, file_path):
        # Read data
        self.data_df = pd.read_csv(file_path)
        self.data_df['target'] = pd.factorize(self.data_df['sentiment'], sort=True)[0]

        # # Split to sentences and labels
        # self.sentences = data['review'].tolist()
        # self.labels = data['sentiment'].tolist()
        #
        # # Enumerate labels
        # self.label_to_idx = {tag: idx for idx, tag in enumerate(sorted(list(set(self.labels))))}
        # self.idx_to_label = {idx: tag for tag, idx in self.label_to_idx.items()}
        #
        # # Tokenize sentences
        # if tokenizer is not None:
        #     self.tokenizer = tokenizer
        #     self.tokenized_sen = self.tokenizer.transform(self.sentences)
        # else:
        #     self.tokenizer = TfidfVectorizer(lowercase=True, stop_words=None)
        #     self.tokenized_sen = self.tokenizer.fit_transform(self.sentences)
        #
        # # Set vocab_size attribute
        # self.vocab_size = len(self.tokenizer.vocabulary_)

    def __getitem__(self, item):
        # Tensorize sentence
        # tokenzied_sentence = self.tokenized_sen[item]
        # tensor_tokenzied_sentence = torch.FloatTensor(tokenzied_sentence.toarray()).squeeze()
        #
        # # Get label idx
        # label = self.labels[item]
        # label = self.label_to_idx[label]
        #
        # # data = {"input_vectors": tensor_tokenzied_sentence, "labels": label}
        # data = label, item
        # print('get_item')
        return self.data_df.iloc[item]['review'], self.data_df.iloc[item]['target'],

    def __len__(self):
        return len(self.data_df)

In [137]:
train_dataset = ClassificationDataset('./data/IMDB_train.csv')
test_dataset = ClassificationDataset('./data/IMDB_test.csv')

In [138]:
next(iter(train_dataset))

("If you enjoy films like American Pie, Road Trip & Van Wilder; avoid this cinematic refuse at all costs. It is an unamusing, mean-spirited, insipid waste of resources that should never have been discussed aloud; much less actually recorded and sold to unsuspecting consumers. Easily the worst film I have seen in the past 18 months; mind-numbingly bad for the entire 86 minutes of it's runtime. Had it been much longer, I would not have been able to write this review without using profanity. Consider yourself warned!",
 0)

In [139]:
train_loader = DataLoader(train_dataset, batch_size=8, collate_fn=vectorize_batch)
test_loader  = DataLoader(test_dataset, batch_size=8, collate_fn=vectorize_batch)

In [140]:
next(iter(train_loader))

(tensor([[ 0.1631,  0.7251,  0.0381,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.4531,  0.2241,  0.0599,  ..., -0.3358,  0.1888, -0.4079],
         [ 0.3689,  0.4753, -0.0480,  ..., -0.5108,  0.4688,  0.3488],
         ...,
         [-0.1304,  0.2049,  0.4257,  ..., -0.4291,  1.0746, -0.3655],
         [ 0.2932,  0.2109,  0.1146,  ..., -0.3128,  0.0414, -0.4936],
         [ 0.1763,  0.3684,  0.3005,  ..., -0.0714, -0.6362,  0.8726]]),
 tensor([0, 1, 1, 0, 0, 1, 0, 1]))

In [141]:
target_classes = ['pos', 'neg']

In [142]:
from torch import nn
from torch.nn import functional as F

class EmbeddingClassifier(nn.Module):
    def __init__(self):
        super(EmbeddingClassifier, self).__init__()
        self.seq = nn.Sequential(
            nn.Linear(max_words*embed_len, 256),
            nn.ReLU(),

            nn.Linear(256,128),
            nn.ReLU(),

            nn.Linear(128,64),
            nn.ReLU(),

            nn.Linear(64, len(target_classes)),
        )

    def forward(self, X_batch):
        return self.seq(X_batch)

In [147]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc

def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())))

def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if i%5==0:
            print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
            CalcValLossAndAccuracy(model, loss_fn, val_loader)

In [148]:
from torch.optim import Adam

epochs = 1
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
embed_classifier = EmbeddingClassifier()
optimizer = Adam(embed_classifier.parameters(), lr=learning_rate)

TrainModel(embed_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

100%|██████████| 5000/5000 [00:36<00:00, 137.80it/s]


In [149]:
def MakePredictions(model, loader):
    Y_shuffled, Y_preds = [], []
    for X, Y in loader:
        preds = model(X)
        Y_preds.append(preds)
        Y_shuffled.append(Y)
    gc.collect()
    Y_preds, Y_shuffled = torch.cat(Y_preds), torch.cat(Y_shuffled)

    return Y_shuffled.detach().numpy(), F.softmax(Y_preds, dim=-1).argmax(dim=-1).detach().numpy()

Y_actual, Y_preds = MakePredictions(embed_classifier, test_loader)

In [92]:
print(embed_classifier)

EmbeddingClassifier(
  (seq): Sequential(
    (0): Linear(in_features=2500, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=2, bias=True)
  )
)
